In [3]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


In [4]:
train_dir = "file location"
labels_file = "file location"


In [6]:
labels_df = pd.read_csv(labels_file)
labels_dict = dict(zip(labels_df.id, labels_df.breed))


In [7]:
image_files = os.listdir(train_dir)
image_paths = []
labels = []

for image_file in image_files:
    image_path = os.path.join(train_dir, image_file)
    image_paths.append(image_path)
    image_id = os.path.splitext(image_file)[0]
    labels.append(labels_dict[image_id])


In [8]:
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


In [9]:
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": train_image_paths, "class": train_labels}),
    directory=None,
    x_col="filename",
    y_col="class",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=pd.DataFrame({"filename": val_image_paths, "class": val_labels}),
    directory=None,
    x_col="filename",
    y_col="class",
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)


Found 8177 validated image filenames belonging to 120 classes.
Found 2045 validated image filenames belonging to 120 classes.


In [10]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(labels_df['breed'].unique()), activation='softmax')
])


In [11]:
optimizer = Adam(learning_rate=0.0001)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [12]:
epochs = 2
history = model.fit(train_generator, epochs=epochs, validation_data=val_generator)


Epoch 1/2
256/256 [==============================] - 258s 992ms/step - loss: 4.7855 - accuracy: 0.0111 - val_loss: 4.7679 - val_accuracy: 0.0152
Epoch 2/2
256/256 [==============================] - 187s 730ms/step - loss: 4.7220 - accuracy: 0.0197 - val_loss: 4.6472 - val_accuracy: 0.0230


In [13]:
# Evaluate the model on the validation set
loss, accuracy = model.evaluate(val_generator)
print("Validation Loss:", loss)
print("Validation Accuracy:", accuracy)


64/64 [==============================] - 10s 157ms/step - loss: 4.6472 - accuracy: 0.0230
Validation Loss: 4.647188186645508
Validation Accuracy: 0.0229828841984272


In [14]:
# Save the model
model.save('dog_breed_model.h5')
